In [49]:
import h5py
import glob
from PIL import Image
import numpy as np
import os
import os.path
import skimage
import skimage.io


In [38]:
def create_hdf(hdf_path):
    hf = h5py.File(hdf_path, 'x')
    hf.create_group('FOV0')
    hf.close()


def tif_2_h5(mask_dir, hdf_path):
    # read tif mask files
    mask_files = sorted(glob.glob(mask_dir + "*.tif"))
    mask_files = [f for f in mask_files if f.endswith('tif')]

    # create a new hdf5 file
    try:
        create_hdf(hdf_path)
    except:
        print("hdf file already exists. overwriting...")
        os.remove(hdf_path)
        create_hdf(hdf_path)

    with h5py.File(hdf_path, 'w') as f:
        for i, mask in enumerate(mask_files):
            im = skimage.io.imread(mask)

            if(len(im.shape) == 3):
                print("3D!!")
            # num pages should be smaller than x or y dimension, very unlikely not to be the case
            # if len(im.shape) == 3 and im.shape[2] < im.shape[0] and im.shape[2] < im.shape[1]:
            #     im = np.moveaxis(im, -1, 0) # move last axis to first

            f.create_dataset('/FOV0/T{}'.format(i),
                             data=im[:, :], compression='gzip')

    f.close()
    return hdf_path


In [39]:
import tifffile


def save_video_to_tif(video_data, output_filename):
    if os.path.exists(output_filename):
        os.remove(output_filename)

    # Open a TIFF file for writing and create a TiffWriter object
    with tifffile.TiffWriter(output_filename) as tif:
        # Iterate over each frame of the video data
        # for frame_idx in range(video_data.shape[0]):
        #     # Make the array contiguous
        #     frame = np.ascontiguousarray(video_data[frame_idx])
        #     # Add the frame as a page to the TIFF file
        # tif.save(frame)
        tif.save(video_data)
    print(f'Saved video to {output_filename}')


def tif_2_stack(tif_dir, stack_filename):
    # Get a list of all TIFF files in the directory
    tif_files = os.listdir(tif_dir)
    tif_files = [f for f in tif_files if f.endswith('.tif')]

    # Sort the file list based on the frame numbers
    tif_files = sorted(tif_files)

    # Load the first frame to get the image size
    img_size = np.array(Image.open(os.path.join(
        tif_dir, tif_files[0])).convert('L')).shape
    stack_volume = np.zeros((len(tif_files),) + img_size, dtype=np.uint16)

    # Load each frame and add it to the stack
    for i, tif_file in enumerate(tif_files):
        img = np.array(Image.open(os.path.join(
            tif_dir, tif_file)))
        stack_volume[i, :, :] = img

    # save to tif
    save_video_to_tif(stack_volume, stack_filename)

    print(f'Saved stack volume to {stack_filename}')


In [43]:
# stack tifs in subset
frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/BF/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/bf_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/mKate/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mkate_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/mKO/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mko_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/segmentation2/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mask_stack.h5'
tif_2_h5(frames_dir, stack_filename)


Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/BF/stacked/bf_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/BF/stacked/bf_stack.tif
Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/mKate/stacked/mkate_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/mKate/stacked/mkate_stack.tif
Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/mKO/stacked/mko_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/mKO/stacked/mko_stack.tif
hdf file already exists. overwriting...


'/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/segmentation2/stacked/mask_stack.h5'

In [44]:
# stack tifs in subset2
frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/BF/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/bf_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/mKate/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mkate_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/mKo/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mko_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/segmentation/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mask_stack.h5'
tif_2_h5(frames_dir, stack_filename)


Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/BF/stacked/bf_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/BF/stacked/bf_stack.tif
Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/mKate/stacked/mkate_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/mKate/stacked/mkate_stack.tif
Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/mKo/stacked/mko_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/mKo/stacked/mko_stack.tif


'/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/segmentation/stacked/mask_stack.h5'

In [ ]:
# stack tifs in subset2
frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/BF/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/bf_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/mKate/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mkate_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/mKo/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mko_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset2/segmentation/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mask_stack.h5'
tif_2_h5(frames_dir, stack_filename)


In [45]:
# stack tifs in training
frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/training/images2/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/bf_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/training/masks2/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mask_stack.h5'
tif_2_h5(frames_dir, stack_filename)


Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/training/images2/stacked/bf_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/training/images2/stacked/bf_stack.tif


'/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/training/masks2/stacked/mask_stack.h5'

In [46]:
# stack tifs in mask and mask2 both in tif and h5
frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/mask2/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/bf_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/mask2/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mask_stack.h5'
tif_2_h5(frames_dir, stack_filename)

# stack tifs in mask and mask2 both in tif and h5
frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/mask/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/bf_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/mask/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mask_stack.h5'
tif_2_h5(frames_dir, stack_filename)


Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/mask2/stacked/bf_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/mask2/stacked/bf_stack.tif
Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/mask/stacked/bf_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/mask/stacked/bf_stack.tif


'/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/mask/stacked/mask_stack.h5'

In [47]:
# stack tifs in images
frames_dir = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/images/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/bf_stack.tif'
tif_2_stack(frames_dir, stack_filename)


Saved video to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/images/stacked/bf_stack.tif
Saved stack volume to /home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/images/stacked/bf_stack.tif


In [48]:
# stack tifs in sc_7
frames_dir = '/home/farzaneh/Documents/sc_7/BF/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/bf_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/sc_7/MKate/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mkate_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/sc_7/MKO/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mko_stack.tif'
tif_2_stack(frames_dir, stack_filename)

frames_dir = '/home/farzaneh/Documents/sc_7/Mask/'
if not os.path.exists(frames_dir+'stacked/'):
    os.makedirs(frames_dir+'stacked/')
stack_filename = frames_dir + 'stacked/mask_stack.h5'
tif_2_h5(frames_dir, stack_filename)


Saved video to /home/farzaneh/Documents/sc_7/BF/stacked/bf_stack.tif
Saved stack volume to /home/farzaneh/Documents/sc_7/BF/stacked/bf_stack.tif
Saved video to /home/farzaneh/Documents/sc_7/MKate/stacked/mkate_stack.tif
Saved stack volume to /home/farzaneh/Documents/sc_7/MKate/stacked/mkate_stack.tif
Saved video to /home/farzaneh/Documents/sc_7/MKO/stacked/mko_stack.tif
Saved stack volume to /home/farzaneh/Documents/sc_7/MKO/stacked/mko_stack.tif


'/home/farzaneh/Documents/sc_7/Mask/stacked/mask_stack.h5'